In [563]:
def createFile(fileName, fileText="""123
4567
8910"""):
    with open(fileName, 'w') as f:
        f.write(fileText)
        
def createFolder(fileName):
    folder = os.path.dirname(fileName)
    if folder and not os.path.exists(folder):
        os.makedirs(folder)
    return

In [564]:
import os
import re
import collections
import chardet
import shutil
import random
import itertools

# Базовый уровень

## Задание 1.1

In [565]:
def copy_file(source_path, target_path):
    folder = os.path.dirname(target_path)
    if folder and not os.path.exists(folder):
        os.makedirs(folder)
    
    with open(source_path, 'r') as fi:
        with open(target_path, 'w') as fo:
            fo.write(fi.read())

    return

In [566]:
def copy_big_file(source_path, target_path):
    createFolder(target_path)
    
    with open(source_path, 'r') as fi, open(target_path, 'w') as fo:
            fo.writelines(line for line in fi.xreadlines())
   
    return

In [567]:
inputFile = "file_in.txt"
outputFile = os.path.join("A","B","out.txt")

createFile(inputFile)

print "\nИсходный файл: ", inputFile
with open(inputFile, 'r') as f:
    print f.read()

copy_file(inputFile, outputFile )

print "\nСкопированный файл: ", outputFile
with open(outputFile, 'r') as f:
    print f.read()

outputFile = os.path.join("A","B","out_big.txt")
copy_big_file(inputFile, outputFile)

print "\nСкопированный файл (построчно): ", outputFile
with open(outputFile, 'r') as f:
    print f.read()


Исходный файл:  file_in.txt
123
4567
8910

Скопированный файл:  A/B/out.txt
123
4567
8910

Скопированный файл (построчно):  A/B/out_big.txt
123
4567
8910


## Задание 1.2

In [568]:
def eng_lat_2_lat_eng(source_path, target_path):
    createFolder(target_path)
    
    ddict = collections.defaultdict(list)
    
    with open(source_path, "r") as fi:
        for line in fi.xreadlines():
            words = re.split("[-,]",line)
            for lat in words[1:]:
                ddict[lat.strip()].append(words[0].strip())
            
    with open(target_path, "w") as fo:
        for word in sorted(ddict.keys()):
            fo.write(word + ' - ' + ', '.join(sorted(ddict[word]))+"\n")
            
    return

In [569]:
inputFile = "file_in.txt"
outputFile = os.path.join("out.txt")

createFile(inputFile,"""apple - malum, pomum, popula
fruit - baca, bacca, popum
punishment - malum, multa""")

eng_lat_2_lat_eng(inputFile, outputFile)

print("Латино-английский словарь:")
with open(outputFile, "r") as f:
    print(f.read())

Латино-английский словарь:
baca - fruit
bacca - fruit
malum - apple, punishment
multa - punishment
pomum - apple
popula - apple
popum - fruit



## Задание 1.3

In [570]:
def my_len(word, encoding=None):
    if type(word) is unicode:
        return len(word)
    
    if encoding==None:
        encoding=chardet.detect(word)['encoding']
    
    return len(word.decode(encoding))

In [571]:
my_len('текст') == 5

True

In [572]:
my_len(u'текст') == 5

True

In [328]:
my_len(u'ааааа'.encode('utf_16'), encoding='utf_16') == 5

True

In [573]:
my_len(u'текст'.encode('windows-1251')) == 5

True

# Продвинутый уровень


## Задание 2.1

In [574]:
def get_dir_content_info(dir_path):
    file_list = os.listdir(dir_path)
    
    res = [(name, os.stat(os.path.join(dir_path,name)).st_size) 
            for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path,name))]
    
    return sorted(res, key=lambda x: (-x[1], x[0]) )

In [575]:
#Создание тестовых файлов
folder = "Files to sort"
if os.path.exists(folder):
    shutil.rmtree(folder)
os.makedirs(folder)
    
for i,x in enumerate("ABCDEFGH"):
    createFile(os.path.join(folder,x+".txt"),"".join([x*int(i%3*10+10)]))

#Получить отсортированный список файлов
print get_dir_content_info(folder)

[('C.txt', 30), ('F.txt', 30), ('B.txt', 20), ('E.txt', 20), ('H.txt', 20), ('A.txt', 10), ('D.txt', 10), ('G.txt', 10)]


## Задание 2.2

In [576]:
def eng_rus_2_rus_eng(source_path, target_path, encoding=None):
    createFolder(target_path)
    
    ddict = collections.defaultdict(list)
    
    with open(source_path, "r") as fi:
        text=fi.read()
        
    if encoding==None:
        encoding=chardet.detect(text)['encoding']
        print "Определна кодировка:{}".format(encoding)
    
    lines=text.decode(encoding).splitlines()

    for line in lines:
        words = re.split("[-,]",line)
        for lat in words[1:]:
            ddict[lat.strip()].append(words[0].strip())
    
    #Результат всегда кодируется в windows-1251
    with open(target_path, "w") as fo:
        for word in sorted(ddict.keys()):
            r = word + u' - ' + u', '.join(sorted(ddict[word])) + u'\n'
            fo.write(r.encode('windows-1251') )
            
    return

In [577]:
inputFile = "file_in.txt"
outputFile = os.path.join("out.txt")
encoding = "cp866"

createFile(inputFile,u"""apple - яблоко, яблоня
cat - кот, кошка
grapnel - кошка""".encode(encoding) )

eng_rus_2_rus_eng(inputFile, outputFile, encoding)
print("Русско-английский словарь (кодировка задана):")
with open(outputFile, "r") as f:
    print(f.read().decode("windows-1251"))
    
eng_rus_2_rus_eng(inputFile, outputFile)
print("Русско-английский словарь (автоопредеоение кодировки):")
with open(outputFile, "r") as f:
    print(f.read().decode("windows-1251"))

Русско-английский словарь (кодировка задана):
кот - cat
кошка - cat, grapnel
яблоко - apple
яблоня - apple

Определна кодировка:IBM866
Русско-английский словарь (автоопредеоение кодировки):
кот - cat
кошка - cat, grapnel
яблоко - apple
яблоня - apple



# Сложный уровень

## Задание 3.1

In [578]:
def sorted_word(word):
    if len(word)>3:
        return word[0]+"".join(sorted(word[1:-1]))+word[-1]
    else:
        return word
    
def random_word(word):
    if len(word)>3:
        mid = list(word[1:-1])
        random.shuffle(mid)
        return word[0]+"".join(mid)+word[-1]
    else:
        return word

def text_shuffle(text, seed=None):
    template = re.compile(u"[а-яА-Я]+|[a-zA-Z]+")
    words = re.findall(template,text)
    
    if seed==None:
        words = map(sorted_word, words)
    else:
        random.seed(seed)
        words = map(random_word, words)
        
    return " ".join(words)

In [579]:
text = u"вам потребуется модуль random и функции random seed и random shuffle"
print text_shuffle(text)
print text_shuffle(text,10)
print text_shuffle(text,20)
print text_shuffle(text,10)


text = "вам потребуется модуль random и функции random seed и random shuffle"
print("\nНа не юникодовских строчках модуль re с шаблоном unicode корректно находит только слова из английских букв, так как коды английских символов в unicode и ASCII совпадают:")
print text_shuffle(text)

print("\nПри строковом шаблоне с русскими буквами модуль re работает некорректно из-за ошибок определния символов в потоке байт:")
template = re.compile("[а-яА-Я]+|[a-zA-Z]+")
print ",".join(re.findall(template,text))

print("\nПри перекодировке строки в unicode получается требуемый результат:")
print text_shuffle(text.decode('utf-8'))

вам пбееорсттуя мдлоуь radnom и фикнуци radnom seed и radnom sffhlue
вам простетбеуя мдлоуь rnadom и фцункии raondm seed и rondam sufhfle
вам побертетуся мдлоуь rndaom и фцкунии rdanom seed и rdanom sulfhfe
вам простетбеуя мдлоуь rnadom и фцункии raondm seed и rondam sufhfle

На не юникодовских строчках модуль re с шаблоном unicode корректно находит только слова из английских букв, так как коды английских символов в unicode и ASCII совпадают:
radnom radnom seed radnom sffhlue

При строковом шаблоне с русскими буквами модуль re работает некорректно из-за ошибок определния символов в потоке байт:
вам,по�,�,еб�,е�,�,я,мод�,л�,random,и,�,�,нк�,ии,random,seed,и,random,shuffle

При перекодировке строки в unicode получается требуемый результат:
вам пбееорсттуя мдлоуь radnom и фикнуци radnom seed и radnom sffhlue


## Задание 3.2

In [580]:
def my_compare(a,b):
    if a>b:
        res = "First"
    elif a<b:
        res = "Second"
    else:
        res = "Equal"
    return "{} vs {} -> {}\n".format(a,b,res)

def process(fst_source_path, snd_source_path, target_path):
    
    with open(fst_source_path, "r") as f1, open(snd_source_path, "r") as f2, open(target_path, "w") as fo:
        lines1 = f1.xreadlines()
        lines2 = f2.xreadlines()
        
        for x, y in itertools.izip_longest(lines1, lines2):
            if x!= None:
                x=x.strip()
            if y!= None:
                y=y.strip()
                
            fo.write(my_compare(x,y))
            
    return

In [581]:
inputFile1 = "file1.txt"
inputFile2 = "file2.txt"
outputFile = os.path.join("3_2", "out.txt")
encoding = "utf-8"

createFile(inputFile1,u"""Ананас
Груша
Виноград
Яблоко""".encode(encoding) )

createFile(inputFile2,u"""Томат
Груша
Огурец""".encode(encoding) )

createFolder(outputFile)

process(inputFile1, inputFile2, outputFile)

with open(outputFile,"r") as f:
    print(f.read().decode(encoding))
    

Ананас vs Томат -> Second
Груша vs Груша -> Equal
Виноград vs Огурец -> Second
Яблоко vs None -> First

